In [47]:
import jieba
import pandas as pd

In [2]:
data = pd.read_csv('./武Dong乾坤.txt', sep='/n', names=['sentence'], encoding='UTF-8')

<ipython-input-2-d062da05d3a5>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('/Users/liuhy/Desktop/武Dong乾坤.txt', sep='/n', names=['sentence'], encoding='UTF-8')


In [3]:
data.head()

,sentence
0,------------
1,第一卷
2,------------
3,第一章 林动
4,第一章林动(本章免费)


In [4]:
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82762 entries, 0 to 82761
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  82762 non-null  object
dtypes: object(1)
memory usage: 646.7+ KB


(82762, 1)

In [5]:
data.loc[6, 'sentence']

'当林动费尽所有的力气睁开那有些沉重的眼皮时，简陋而整洁的房间顿时出现在眼中，熟悉的一幕让得他愣了愣，旋即连忙转头，果然是见到，在那房中，一男一女两道身影正坐在桌旁。'

### 数据处理--将文本分词，封为一个list供训练使用

In [6]:
jieba.load_userdict('./my_dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/6m/0qpv989x76191k9prmny3p3w0000gn/T/jieba.cache
Loading model cost 0.653 seconds.
Prefix dict has been built successfully.


In [7]:
stop_words = [' ', '(', ')', '（', '）', ',', '.', '!', '?', '|', '，', '。', '！', '？', '|', '｜', '……', '"', '"', '“', '”', "'", '‘', '’', '…', '------------']    

In [8]:
data['sentence'] = data['sentence'].apply(lambda sent: [word for word in jieba.lcut(sentence=sent, cut_all=False, HMM=True) if word not in stop_words])       

In [9]:
data.head(10)

,sentence
0,[]
1,[第一卷]
2,[]
3,"[第一章, 林动]"
4,"[第一章, 林动, 本章, 免费]"
5,[唔]
6,"[当, 林动, 费尽, 所有, 的, 力气, 睁开, 那, 有些, 沉重, 的, 眼皮, 时..."
7,"[爹, 娘]"
8,"[望, 着, 那, 两道, 身影, 林动, 赶忙, 提起, 精神, 小声, 的, 道]"
9,"[动儿, 你, 醒, 了]"


In [14]:
sents_list = [sent for sent in list(data['sentence']) if len(sent) > 4]

### 使用gensim完成word2vec

In [17]:
# !pip install gensim

In [18]:
from gensim.models import fasttext
from gensim.models import word2vec
import logging

In [19]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = word2vec.Word2Vec(sentences=sents_list, window=5, sg=0, min_count=1, negative=5, epochs=20)     

2022-03-28 23:30:07,793 : INFO : collecting all words and their counts
2022-03-28 23:30:07,794 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-28 23:30:07,846 : INFO : PROGRESS: at sentence #10000, processed 317114 words, keeping 17544 word types
2022-03-28 23:30:07,903 : INFO : PROGRESS: at sentence #20000, processed 661903 words, keeping 24491 word types
2022-03-28 23:30:07,959 : INFO : PROGRESS: at sentence #30000, processed 991328 words, keeping 30783 word types
2022-03-28 23:30:08,015 : INFO : PROGRESS: at sentence #40000, processed 1296874 words, keeping 35557 word types
2022-03-28 23:30:08,065 : INFO : PROGRESS: at sentence #50000, processed 1585353 words, keeping 38994 word types
2022-03-28 23:30:08,118 : INFO : PROGRESS: at sentence #60000, processed 1883132 words, keeping 42564 word types
2022-03-28 23:30:08,158 : INFO : collected 45067 word types from a corpus of 2082846 raw words and 66905 sentences
2022-03-28 23:30:08,159 : INFO : Creatin

2022-03-28 23:30:20,806 : INFO : EPOCH - 10 : training on 2082846 raw words (1726857 effective words) took 1.1s, 1543626 effective words/s
2022-03-28 23:30:21,814 : INFO : EPOCH 11 - PROGRESS: at 84.23% examples, 1468988 words/s, in_qsize 5, out_qsize 0
2022-03-28 23:30:21,982 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-28 23:30:21,986 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-28 23:30:21,988 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-28 23:30:21,988 : INFO : EPOCH - 11 : training on 2082846 raw words (1726759 effective words) took 1.2s, 1467943 effective words/s
2022-03-28 23:30:23,002 : INFO : EPOCH 12 - PROGRESS: at 83.73% examples, 1452414 words/s, in_qsize 5, out_qsize 0
2022-03-28 23:30:23,162 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-28 23:30:23,165 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-28 23:30:23,166 : INFO : work

In [20]:
model.save('./word2vec.model')

2022-03-28 23:35:34,607 : INFO : Word2Vec lifecycle event {'fname_or_handle': './word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-28T23:35:34.607005', 'gensim': '4.1.2', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'saving'}
2022-03-28 23:35:34,608 : INFO : not storing attribute cum_table
2022-03-28 23:35:34,675 : INFO : saved ./word2vec.model


In [29]:
word_vectors = model.wv

In [37]:
word_vectors.save('./word2vec_WDQK.wordvectors')

2022-03-28 23:54:57,213 : INFO : KeyedVectors lifecycle event {'fname_or_handle': './word2vec_WDQK.wordvectors', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-28T23:54:57.213911', 'gensim': '4.1.2', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'saving'}
2022-03-28 23:54:57,251 : INFO : saved ./word2vec_WDQK.wordvectors


In [1]:
from gensim.models import keyedvectors

In [2]:
word_vectors_kv = keyedvectors.KeyedVectors.load('./word2vec_WDQK.wordvectors')

In [3]:
type(word_vectors_kv.key_to_index)

dict

In [27]:
words = list(word_vectors_kv.key_to_index.keys())
vectors = list(word_vectors_kv.vectors)
vectors = list(map(list, (vec for vec in vectors)))

In [28]:
len(words), len(vectors)

(45067, 45067)

In [65]:
str_vectors = list(map(lambda x: ' '.join(x), [[str(vec_value) for vec_value in vec] for vec in vectors]))     

In [66]:
len(str_vectors)

45067

In [67]:
word_vectors_dict = list(zip(words, str_vectors))

In [68]:
embedding = pd.DataFrame(word_vectors_dict, columns=['word', 'vector'])

In [69]:
embedding.head()

,word,vector
0,的,-0.5071854 0.43013474 0.9458973 -0.46390238 -1...
1,是,-1.8130685 -0.5626644 -0.14609273 2.3658862 1....
2,了,0.5621393 1.2186996 0.55153453 -1.5065137 -1.6...
3,林动,-1.7170911 0.071835935 0.3490898 -1.7563043 -0...
4,在,0.6984693 1.3269782 -0.75043553 0.75340396 0.0...


In [76]:
embedding.to_csv('./embedding.txt',index=False,header=False)